In [ ]:
# Install the tdmclient package:
!pip3 install tdmclient --upgrade
!pip3 install numpy --upgrade

import tdmclient.notebook
import numpy as np
import math
from tdmclient import ClientAsync, aw

client = ClientAsync()

# Physical constants of the Thymio
WHEELS_DIST_MM = 95.0
SPEED_MM_S = 200.0/500.0 #Calibrate per robot

def set_motors(node, left, right):
    aw(node.set_variables({"motor.left.target": [int(left)],"motor.right.target": [int(right)]}))

def turn_in_place(node, delta_theta, speed=150, dt=0.01, tolerance=0.02):
    if delta_theta > 0:
        direction = 1
    else:
        direction = -1

    set_motors(node, -direction * speed, direction * speed)

    current_theta = 0.0

    # Wait for motor speed to be available 
    aw(node.wait_for_variables({"motor.left.speed", "motor.right.speed"}))

    # Integrate until we reach the desired angle
    while abs(delta_theta) - abs(current_theta) > tolerance:
        aw(client.sleep(dt))

        v_l = node["motor.left.speed"]
        v_r = node["motor.right.speed"]

        # mm travelled over dt
        dl = v_l * SPEED_MM_S * dt
        dr = v_r * SPEED_MM_S * dt

        # Increment heading change
        dtheta = (dr - dl) / WHEELS_DIST_MM
        current_theta += dtheta
    set_motors(node, 0, 0)

def move_straight(node, distance_mm, speed=150, dt=0.01):

    if distance_mm > 0:
        direction = 1
    else:
        direction = -1
    
    set_motors(node, direction * speed, direction * speed)

    travelled = 0.0
    aw(node.wait_for_variables({"motor.left.speed", "motor.right.speed"}))

    while abs(travelled) < abs(distance_mm):
        aw(client.sleep(dt))

        v_l = node["motor.left.speed"]
        v_r = node["motor.right.speed"]

        # Distance travelled by the centre of the robot in this interval
        dl = v_l * SPEED_MM_S * dt
        dr = v_r * SPEED_MM_S * dt
        dcenter = 0.5 * (dl + dr)

        travelled += dcenter

    set_motors(node, 0, 0)

def move_to_position_odometry(node, target_x_mm, target_y_mm,x=0.0, y=0.0, theta=0.0,
                              linear_speed=150, angular_speed=150,
                              dt=0.01, pos_tol=5.0, ang_tol=0.02):
    ## ----> y axis
    #|
    #|
    #|
    #x axis

    # Vector from current pose to target
    dx = target_x_mm - x
    dy = target_y_mm - y

    # Angle from +y (forward) to the target (pos is left)
    desired_theta = math.atan2(-dx, dy)

    #Normalize to [–pi, pi]
    delta_theta = desired_theta - theta
    delta_theta = math.atan2(math.sin(delta_theta), math.cos(delta_theta))

    # Rotate in place to point towards the target
    turn_in_place(node, delta_theta,
                  speed=angular_speed, dt=dt, tolerance=ang_tol)
    theta = desired_theta  #We assume the robot was able to rotate

    # Drive forward while integrating odometry
    set_motors(node, linear_speed, linear_speed)
    aw(node.wait_for_variables({"motor.left.speed", "motor.right.speed"}))

    while True:
        aw(client.sleep(dt))

        v_l = node["motor.left.speed"]
        v_r = node["motor.right.speed"]

        dl = v_l * SPEED_MM_S * dt
        dr = v_r * SPEED_MM_S * dt
        dcenter = 0.5 * (dl + dr)
        dtheta = (dr - dl) / WHEELS_DIST_MM

        theta += dtheta
        x += -dcenter * math.sin(theta)
        y +=  dcenter * math.cos(theta)

        dx = target_x_mm - x
        dy = target_y_mm - y

        if math.hypot(dx, dy) <= pos_tol:
            break

    set_motors(node, 0, 0)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
# Get a node and lock it (might not be needed for the final implementation)
node = aw(client.wait_for_node())
aw(node.lock())

try:
    move_to_position_odometry(node, 10, 0)
finally:
    #Makes sure the robot is stopped and the node available
    aw(node.set_variables({
        "motor.left.target": [0],
        "motor.right.target": [0]}))
    aw(node.stop())
    aw(node.unlock())